In [3]:
import pandas
import os
import csv

%matplotlib inline

In [14]:
out_path = r'C:\Users\pechen\Desktop\VTP\Model_Analysis\20191211_KK\Microsim Results'
in_path = r'M:\VaughanTMP\Input_Directory_Disagg\Scenario-Population\2016\20191202_Brendan\ZoneData'

calib_dir = r'C:\Users\pechen\Desktop\VTP\Model_Analysis\Calib_Check'

In [15]:
pd_df = pandas.read_csv(os.path.join(in_path, "PDs.csv"))
pd_df.head()

,Zone#,PD
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [16]:
hhld_df = pandas.read_csv(os.path.join(out_path, "households.csv"))
hhld_df = hhld_df.rename(columns ={'weight': 'hhld_weight'})
hhld_df = pandas.merge(hhld_df,pd_df, left_on = 'home_zone', right_on = 'Zone#', how = 'left')
hhld_df = hhld_df.rename(columns ={'PD': 'home_PD'})
hhld_df = hhld_df.drop("Zone#", axis = 1)

In [17]:
#Generate Auto Ownership - GTHA
hhld_df['vehicles'].value_counts(sort = False)

0    393554
1    992601
2    817291
3    199325
4     64369
Name: vehicles, dtype: int64

In [19]:
#Generate Auto Ownership
hhld_df[hhld_df['home_PD']==33]['vehicles'].value_counts(sort = False)

2    44603
1    20378
3    13451
4     7491
0     3704
Name: vehicles, dtype: int64

In [20]:
# load persons file
person_df = pandas.read_csv(os.path.join(out_path, "persons.csv"))
person_df = person_df.rename(columns ={'weight': 'person_weight'})
hhld_person_df = pandas.merge(hhld_df,person_df, on =['household_id'] )
hhld_person_df = pandas.merge(hhld_person_df,pd_df, left_on = 'work_zone', right_on = 'Zone#', how = 'left')
hhld_person_df = hhld_person_df.rename(columns ={'PD': 'work_PD'})
hhld_person_df = hhld_person_df.drop("Zone#", axis = 1)

In [21]:
# load mode data
trip_df =  pandas.read_csv(os.path.join(out_path, "trips.csv"))
trip_df = trip_df.rename(columns ={'weight': 'trip_weight'})
mode_df = pandas.read_csv(os.path.join(out_path, "trip_modes.csv"))
mode_df = mode_df.rename(columns ={'weight': 'mode_weight'})

In [22]:
# generate trip mode df
trip_mode_df = pandas.merge(trip_df,mode_df, on =['household_id','person_id','trip_id'],how = 'outer' )

trip_mode_df = pandas.merge(trip_mode_df,pd_df, left_on = 'o_zone', right_on = 'Zone#', how = 'left')
trip_mode_df = trip_mode_df.rename(columns ={'PD': 'o_PD'})
trip_mode_df = trip_mode_df.drop("Zone#", axis = 1)

trip_mode_df = pandas.merge(trip_mode_df,pd_df, left_on = 'd_zone', right_on = 'Zone#', how = 'left')
trip_mode_df = trip_mode_df.rename(columns ={'PD': 'd_PD'})
trip_mode_df = trip_mode_df.drop("Zone#", axis = 1)

In [23]:
person_trip_df = pandas.merge(hhld_person_df,trip_mode_df, on =['household_id','person_id'])

In [24]:
#AM Peak
time_start = 360
time_end = 540
od_file_name = "OD_AM.csv"

# #PM Peak 
# time_start = 900
# time_end = 1140
# od_file_name = "OD_PM.csv"

# #Daily
# time_start = 0
# time_end = 2000
# od_file_name = "OD_daily.csv"

In [25]:
# only 11+ yr olds
time_period_df = person_trip_df[(person_trip_df['age'] >10) \
                                & (person_trip_df['o_depart'] >= time_start) \
                                & (person_trip_df['o_depart']< time_end )]

In [72]:

pd_mode_pt = pandas.pivot_table(time_period_df, values = "mode_weight", index =['o_PD','d_PD','mode'],aggfunc = sum).reset_index()
with open (os.path.join(calib_dir,od_file_name), 'w') as od_file:
    od_writer = csv.writer(od_file, dialect='excel')
    
    PD_list = [[33],
               range(1,17),
               range(25,33),
               range(34,37),
               chain(range(0), range(17,25), range(37,47))
               ]
    mode_list =[['Auto'],
                ['Passenger','Rideshare','Carpool'],
                ['VFH'],
                ['DAT','PAT','PET'],
                ['WAT'],
                ['Bicycle','Walk'],
                ['Schoolbus']
                ]
    
    for o_PDs in PD_list:
        for d_PDs in PD_list:
            #only to or from Vaughan
            if o_PDs == [33] or d_PDs == [33]:
                mode_row = []
                for od_modes in mode_list:
                    total_trips = 0.0
                    for o_PD in o_PDs:
                        for d_PD in d_PDs:
                            print(str(o_PD) + " to" + str(d_PD))
                            #print(total_trips)
                            for od_mode in od_modes:      
                                total_trips += pd_mode_pt[(pd_mode_pt['o_PD'] == o_PD)\
                                           & (pd_mode_pt['d_PD'] == d_PD)\
                                           & (pd_mode_pt['mode'] == od_mode)].sum()['mode_weight']/10.0

                    mode_row.append(total_trips)
                # od_writer.writerow(o_PDs)
                # od_writer.writerow(d_PDs)
                od_writer.writerow(mode_row)
            

33 to33
33 to33
33 to33
33 to33
33 to33
33 to33
33 to33
33 to1
33 to2
33 to3
33 to4
33 to5
33 to6
33 to7
33 to8
33 to9
33 to10
33 to11
33 to12
33 to13
33 to14
33 to15
33 to16
33 to1
33 to2
33 to3
33 to4
33 to5
33 to6
33 to7
33 to8
33 to9
33 to10
33 to11
33 to12
33 to13
33 to14
33 to15
33 to16
33 to1
33 to2
33 to3
33 to4
33 to5
33 to6
33 to7
33 to8
33 to9
33 to10
33 to11
33 to12
33 to13
33 to14
33 to15
33 to16
33 to1
33 to2
33 to3
33 to4
33 to5
33 to6
33 to7
33 to8
33 to9
33 to10
33 to11
33 to12
33 to13
33 to14
33 to15
33 to16
33 to1
33 to2
33 to3
33 to4
33 to5
33 to6
33 to7
33 to8
33 to9
33 to10
33 to11
33 to12
33 to13
33 to14
33 to15
33 to16
33 to1
33 to2
33 to3
33 to4
33 to5
33 to6
33 to7
33 to8
33 to9
33 to10
33 to11
33 to12
33 to13
33 to14
33 to15
33 to16
33 to1
33 to2
33 to3
33 to4
33 to5
33 to6
33 to7
33 to8
33 to9
33 to10
33 to11
33 to12
33 to13
33 to14
33 to15
33 to16
33 to25
33 to26
33 to27
33 to28
33 to29
33 to30
33 to31
33 to32
33 to25
33 to26
33 to27
33 to28
33 to29
33 to30

,o_PD,d_PD,mode,mode_weight
0,0,0,Auto,104
1,0,0,Carpool,2
2,0,0,Schoolbus,14
3,0,0,VFH,15
4,0,0,Walk,38
...,...,...,...,...
13317,46,46,RideShare,3636
13318,46,46,Schoolbus,46735
13319,46,46,VFH,26012
13320,46,46,WAT,135854
